In [1]:
from collections import namedtuple
# other packages
import pandas as pd
from allennlp.predictors import SimpleSeq2SeqPredictor, Seq2SeqPredictor
import tqdm
# pytorch
import torch
import torch.optim as optim
# preprocessing
from imblearn.under_sampling import RandomUnderSampler
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.vocabulary import Vocabulary
# model
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
# attention
from allennlp.modules.attention import DotProductAttention
# encoder
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.training.trainer import Trainer
from sklearn.model_selection import train_test_split




In [2]:
torch.cuda.is_available()

True

In [3]:


Config = namedtuple('Config', [
                    'lazy',
                    'max_vocab_size', 
                    'batch_size', 
                    'epochs', 
                    'max_seq_len', 
                    'IN_EMBEDDING_DIM', 
                    'HIDDEN_DIM', 
                    'OUT_EMBEDDING_DIM', 
                    'CUDA_DEVICE',
                    'n_samples'
])



# create config file

In [4]:
config = Config(False,
               10000,
               64,
               10,
               100,
               64,
               32,
               64,
               0 if torch.cuda.is_available() else -1, 
               60000)

# prepare data

In [5]:
raw_dataset = pd.read_csv('data/ru_train.csv')

raw_dataset['before'] = raw_dataset['before'].astype(str)
raw_dataset['after'] = raw_dataset['after'].astype(str)

d = raw_dataset['class'].value_counts().to_dict()

for i in d:
    if d[i] > config.n_samples:
        d[i] = config.n_samples

In [6]:
d

{'PLAIN': 60000,
 'PUNCT': 60000,
 'CARDINAL': 60000,
 'LETTERS': 60000,
 'DATE': 60000,
 'VERBATIM': 60000,
 'ORDINAL': 46738,
 'MEASURE': 40534,
 'TELEPHONE': 10088,
 'DECIMAL': 7297,
 'ELECTRONIC': 5832,
 'MONEY': 2690,
 'FRACTION': 2460,
 'DIGIT': 2012,
 'TIME': 1945}

In [7]:


rus = RandomUnderSampler(sampling_strategy=d, random_state=0)

raw_dataset_resampled, _ = rus.fit_resample(raw_dataset, raw_dataset['class'])



In [8]:
df = raw_dataset_resampled[['before', 'after']]

df_train, df_test = train_test_split(df, test_size=0.1)

df_train.to_csv('train_dataset.tsv', index=False, header=False, sep='\t')
df_test.to_csv('test_dataset.tsv', index=False, header=False, sep='\t')

# create reader

In [9]:
reader = Seq2SeqDatasetReader(
    source_tokenizer = CharacterTokenizer(),
    target_tokenizer = CharacterTokenizer(),
    source_token_indexers={'tokens': SingleIdTokenIndexer()},
    target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')},
    lazy=config.lazy
)

train_dataset = reader.read('train_dataset.tsv',)
validation_dataset = reader.read('test_dataset.tsv')

431636it [00:25, 17100.20it/s]
47960it [00:01, 32486.35it/s]


# prepare vocabulary

In [10]:
vocab = Vocabulary.from_instances(train_dataset,
                                  min_count={'tokens': 3, 'target_tokens': 3}
                                 )

in_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                         embedding_dim=config.IN_EMBEDDING_DIM
                        )

source_embedder = BasicTextFieldEmbedder({"tokens": in_embedding})

iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("source_tokens", "num_tokens")],
                         )

iterator.index_with(vocab)

100%|██████████| 431636/431636 [00:06<00:00, 69163.62it/s]


# prepare model

In [11]:
## get simple encoder
encoder = PytorchSeq2SeqWrapper(torch.nn.LSTM(config.IN_EMBEDDING_DIM, 
                                              config.HIDDEN_DIM, 
                                              batch_first=True))

attention = DotProductAttention()

In [12]:
max_decoding_steps = 100

model = SimpleSeq2Seq(vocab, 
                      source_embedder, 
                      encoder, 
                      max_decoding_steps,
                      target_embedding_dim=config.OUT_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      beam_size=8,
                      use_bleu=True,
                      attention=attention,
                      scheduled_sampling_ratio = 0.15)

In [13]:
if torch.cuda.is_available():
    model.cuda(config.CUDA_DEVICE)

In [14]:
optimizer = optim.Adam(model.parameters())

# training

In [15]:
torch.cuda.is_available()

True

In [16]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  num_epochs=1,
                  cuda_device=config.CUDA_DEVICE,
                  patience=2)

In [17]:
print(f'Will train for {config.epochs} epochs')
for i in range(config.epochs):
    print(f'Epoch: {i+1}')
    trainer.train()

  0%|          | 0/6745 [00:00<?, ?it/s]

Will train for 10 epochs
Epoch: 1


loss: 1.6100 ||: 100%|██████████| 6745/6745 [08:27<00:00, 13.28it/s]
  0%|          | 0/750 [00:00<?, ?it/s]/opt/conda/conda-bld/pytorch_1587428398394/work/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
BLEU: 0.4438, loss: 0.7338 ||: 100%|██████████| 750/750 [04:54<00:00,  2.55it/s]
  0%|          | 0/6745 [00:00<?, ?it/s]

Epoch: 2


loss: 0.5348 ||: 100%|██████████| 6745/6745 [08:14<00:00, 13.65it/s]
BLEU: 0.6123, loss: 0.2941 ||: 100%|██████████| 750/750 [04:39<00:00,  2.69it/s]
  0%|          | 0/6745 [00:00<?, ?it/s]

...

loss: 0.1096 ||: 100%|██████████| 6745/6745 [08:17<00:00, 13.55it/s]
BLEU: 0.8629, loss: 0.0800 ||: 100%|██████████| 750/750 [04:08<00:00,  3.02it/s]
  0%|          | 0/6745 [00:00<?, ?it/s]

Epoch: 9


loss: 0.1025 ||: 100%|██████████| 6745/6745 [08:17<00:00, 13.57it/s]
BLEU: 0.8685, loss: 0.0767 ||: 100%|██████████| 750/750 [04:12<00:00,  2.98it/s]
  0%|          | 0/6745 [00:00<?, ?it/s]

Epoch: 10


loss: 0.0992 ||: 100%|██████████| 6745/6745 [08:14<00:00, 13.65it/s]
BLEU: 0.8745, loss: 0.0732 ||: 100%|██████████| 750/750 [04:08<00:00,  3.02it/s]


In [18]:
with open("AdditiveAttention.th", 'wb') as f:
    torch.save(model.state_dict(), f)
    
tqdm.tqdm.pandas()

predictor = SimpleSeq2SeqPredictor(model, reader)

del raw_dataset, df, train_dataset, trainer

/home/analekseev/anaconda3/envs/NLPHW3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [19]:


kaggle_test = pd.read_csv('data/ru_test_2.csv')
kaggle_test['before'] = kaggle_test['before'].astype(str)

small_kaggle = pd.DataFrame(kaggle_test['before'].unique(), columns=['before'])


In [ ]:
small_kaggle['after'] = small_kaggle['before'].progress_apply(lambda x: ''.join(predictor.predict(x.lower())['predicted_tokens']))

 81%|████████  | 141712/175991 [55:09<11:18, 50.51it/s]  

In [ ]:
small_kaggle = small_kaggle.set_index('before')

In [ ]:
kaggle_test = kaggle_test.join(small_kaggle, on='before')
kaggle_test['id'] = kaggle_test['sentence_id'].astype(str)+'_'+kaggle_test['token_id'].astype(str)

In [ ]:
kaggle_test[['id', 'after']].to_csv('sub.csv', index=False)